In [4]:
import pandas as pd

# Function to process data for a specific aircraft type
def process_flight_data(filepath, aircraft_type):
    #Load the dataset
    df = pd.read_pickle(filepath)
    
    #Filter for timestep == 0
    df_filtered = df[df['timestep'] == 0].copy()
    
    #Extract lat and lon, round to 1 decimal place
    df_filtered['lat_rounded'] = df_filtered['lat'].round(1).astype(str).str.replace('.', ',')
    df_filtered['lon_rounded'] = df_filtered['lon'].round(1).astype(str).str.replace('.', ',')
    
    #Count occurrences of each (lat, lon) pair
    coordinate_counts = df_filtered[['lat_rounded', 'lon_rounded']].value_counts().reset_index(name='count')
    coordinate_counts.columns = ['lat_rounded', 'lon_rounded', 'count']
    
    #Export the results to a CSV file with semicolon separator and comma as decimal separator
    output_filename = f"{aircraft_type}_coordinate_counts.csv"
    
    try:
        coordinate_counts.to_csv(output_filename, sep=';', index=False)
        print(f"Results saved to {output_filename}")
    except OSError as e:
        print(f"Failed to save the file: {e}")

#Call the function
process_flight_data("/Users/paulkloos/Desktop/ERASMUS/TIL_Python/Project/AC_Data/A319.pkl", "A319")
process_flight_data("/Users/paulkloos/Desktop/ERASMUS/TIL_Python/Project/AC_Data/A320.pkl", "A320")
process_flight_data("/Users/paulkloos/Desktop/ERASMUS/TIL_Python/Project/AC_Data/A321.pkl", "A321")
process_flight_data("/Users/paulkloos/Desktop/ERASMUS/TIL_Python/Project/AC_Data/A332.pkl", "A332")

Results saved to A319_coordinate_counts.csv
Results saved to A320_coordinate_counts.csv
Results saved to A321_coordinate_counts.csv
Results saved to A332_coordinate_counts.csv


In [8]:
import pandas as pd

# Load the CSV files for the four aircraft types
df_a319 = pd.read_csv("A319_coordinate_counts.csv", sep=';')
df_a320 = pd.read_csv("A320_coordinate_counts.csv", sep=';')
df_a321 = pd.read_csv("A321_coordinate_counts.csv", sep=';')
df_a332 = pd.read_csv("A332_coordinate_counts.csv", sep=';')  # Renamed to A332

# Rename columns to represent the occurrence counts for each aircraft type
df_a319.rename(columns={'count': 'count_a319'}, inplace=True)
df_a320.rename(columns={'count': 'count_a320'}, inplace=True)
df_a321.rename(columns={'count': 'count_a321'}, inplace=True)
df_a332.rename(columns={'count': 'count_a332'}, inplace=True)

# Merge the four datasets based on the coordinates (lat_rounded and lon_rounded)
combined = df_a319.merge(df_a320, on=['lat_rounded', 'lon_rounded'], how='outer') \
                  .merge(df_a321, on=['lat_rounded', 'lon_rounded'], how='outer') \
                  .merge(df_a332, on=['lat_rounded', 'lon_rounded'], how='outer')

# Fill missing values with 0, since not all coordinates are present in each dataset
combined.fillna(0, inplace=True)

# Convert the counts to integers
combined[['count_a319', 'count_a320', 'count_a321', 'count_a332']] = combined[['count_a319', 'count_a320', 'count_a321', 'count_a332']].astype(int)

# Add a condition: Keep only the rows where all counts are >= 15
filtered_combined = combined[(combined['count_a319'] >= 15) & 
                             (combined['count_a320'] >= 15) & 
                             (combined['count_a321'] >= 15) & 
                             (combined['count_a332'] >= 15)]

# Display the result
print("Coordinates flown by all four aircraft types with at least 15 occurrences:")
print(filtered_combined)

#Save the result to a CSV file
filtered_combined.to_csv("filtered_all_types_min_15_coordinates.csv", sep=';', index=False

Coordinates which have been flown by minimum 15 aircrafts:
      lat_rounded lon_rounded  count_a319  count_a320  count_a321  count_a332
275         -23,4       -46,4          65        1563        1014         278
276         -23,4       -46,5          61         682         424          84
307         -23,6       -46,5          20         246         104          20
818         -34,8       -56,0          67         218          49          35
826         -34,8       -58,5          40          63          40         139
...           ...         ...         ...         ...         ...         ...
12652        55,6        37,3         117          51          42         115
12750        55,8        -4,6          59         136          91          31
12782        55,9        -3,4         875        1194         132          33
13278        60,3        11,1          77         149          44          16
13291        60,3        25,0         659         980        1166          16

[97 